In [ ]:
# default_exp core

In [1]:
#hide
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
#hide
!pip install nbdev
from nbdev.showdoc import *
ROOT = "/content/drive/MyDrive/Coding/ModelAssistedLabel"

     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 


In [3]:
#hide
!pip install nbdev
import os
# define pathway to the weights
weight_filenames = {
    "lcd": "21-2-20-94-universal-lcd.pt",
    "digits":'21-2-25 1k-digits YOLOv5-weights.pt'
    }

resource_folder = "/content/drive/MyDrive/Coding/Roboflow/try it out"

# detectors = []
# for filename in weight_filenames:
# weights_path = os.path.join(resource_folder, weights_filename)



In [4]:
import os
!cd "{root}"
# safety for re-executions
if not os.path.exists("yolov5"):
  # clone YOLOv5 and reset to a specific git checkpoint that has been verified working
  !git clone https://github.com/ultralytics/yolov5  # clone repo
  !git reset --hard 68211f72c99915a15855f7b99bf5d93f5631330f

# enter the yolov5 directory
%cd yolov5

# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

clear_output()

if torch.cuda.is_available():
  print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0)))
else:
  raise Exception("You need to enable GPU in this runtime environment")

# return to parent directory
%cd ..

Setup complete. Using torch 1.8.0+cu101 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', major=7, minor=0, total_memory=16160MB, multi_processor_count=80)
/content
